# Output Parser


In [1]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [30]:
schema_gift = ResponseSchema(
    name="gift",
    type='bool',
    description="Was the item  bought by someone? true if yes, otherwise or if there is not this info false"
)
schema_delivery = ResponseSchema(
    name="delivery_time",
    type='int',
    description="Time spent to deliver the gift. If the information is missing return -1"
)
schema_perception = ResponseSchema(
    name="value_perception",
    type='list',
    description="Extract a python list with any phrase about the product value or price"
)

In [31]:
output_parser = StructuredOutputParser.from_response_schemas([schema_gift, schema_delivery, schema_perception])

In [6]:
print(output_parser.get_format_instructions())

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": bool  // Was the item  bought by someone? True if yes, otherwise or if there is not this info False
	"delivery_time": int  // Time spent to deliver the gift. If the information is missing return -1
	"value_perception": list  // Extract a python list with any phrase about the product value or price
}
```


In [7]:
review_client = """This leaf blower is pretty amazing. It has four settings:
candle blow, gentle breeze, windy city, and tornado. It arrived in two days,
just in time for my wife's birthday present. I think my wife liked it so much
that she was speechless. So far, I've been the only one to use it, and I've
been using it every other morning to clean the leaves off our lawn. It's a bit
more expensive than other leaf blowers available on the market, but I think
it's worth it for the extra features."""


In [36]:
from langchain.prompts import ChatPromptTemplate

review_template = ChatPromptTemplate.from_template("""
For the following text, extract the following informations:
gift, delivery_time, value_perception
                                                   
Text: {review}
                                                   
(The output must be only (and only) the json with the extracted informations, don't return anything but it)
{schema}
""", partial_variables={'schema': output_parser.get_format_instructions()})


print(review_template.format_messages(review = review_client))

[HumanMessage(content='\nFor the following text, extract the following informations:\ngift, delivery_time, value_perception\n                                                   \nText: This leaf blower is pretty amazing. It has four settings:\ncandle blow, gentle breeze, windy city, and tornado. It arrived in two days,\njust in time for my wife\'s birthday present. I think my wife liked it so much\nthat she was speechless. So far, I\'ve been the only one to use it, and I\'ve\nbeen using it every other morning to clean the leaves off our lawn. It\'s a bit\nmore expensive than other leaf blowers available on the market, but I think\nit\'s worth it for the extra features.\n                                                   \n(The output must be only (and only) the json with the extracted informations, don\'t return anything but it)\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"gift"

In [37]:
from langchain_ollama import ChatOllama

chat = ChatOllama(model="llama2")
answer = chat.invoke(review_template.format_messages(review = review_client))
print(answer.content)

```json
{
	"gift": True,
	"delivery_time": 2,
	"value_perception": ["it's worth it for the extra features", "bit more expensive than other leaf blowers available on the market"]
}
```


In [40]:
print(answer.content)


```json
{
	"gift": True,
	"delivery_time": 2,
	"value_perception": ["it's worth it for the extra features", "bit more expensive than other leaf blowers available on the market"]
}
```


In [ ]:
output_parser.parse(answer.content)

### Pydantic

In [38]:
review_cliente = """Este soprador de folhas é bastante incrível. Ele tem 
quatro configurações: sopro de vela, brisa suave, cidade ventosa 
e tornado. Chegou em dois dias, bem a tempo para o presente de 
aniversário da minha esposa. Acho que minha esposa gostou tanto 
que ficou sem palavras. Até agora, fui o único a usá-lo, e tenho 
usado em todas as manhãs alternadas para limpar as folhas do 
nosso gramado. É um pouco mais caro do que os outros sopradores 
de folhas disponíveis no mercado, mas acho que vale a pena pelas 
características extras."""

In [39]:
from pydantic import BaseModel, Field

class AvaliacaoReview(BaseModel):
    """Avalia review do cliente"""
    presente: bool = Field(description='Verdadeiro se foi para presente e False se não foi')
    dias_entrega: int = Field(description='Quantos dias para entrega do produto')
    percepcao_valor: list[str] = Field(description='Extraia qualquer frase sobre o valor ou \
    preço do produto. Retorne uma lista.')

llm_estruturada = chat.with_structured_output(AvaliacaoReview)
resposta = llm_estruturada.invoke(review_cliente)
resposta

AvaliacaoReview(presente=True, dias_entrega=2, percepcao_valor=['basta', 'valor_justificado'])